Edited by Taylor and working on Thursday, Sept 7th 2023

In [13]:
# key parameters
gcpt_dash_file = 'GCPT dashboard data 2023-08.xlsx'
gcpt_dash_path = '/Users/gem-tah/Desktop/GEM_INFO/GEM_WORK/GOGPT-Dashboard/data/'

# max_year = 2022

In [14]:
import pandas as pd
import numpy as np
import math
import time
from pandas.api.types import CategoricalDtype
import gspread
import os

In [15]:
max_year = 2023

client_secret = "Desktop/GEM_INFO/client_secret.json"
client_secret_full_path = os.path.expanduser("~/") + client_secret

gcpt_dash_file = 'GCPT dashboard data 2023-08.xlsx'
gcpt_dash_path = os.path.expanduser('/Users/gem-tah/Desktop/GEM_INFO/GEM_WORK/GCPT-dashboard/data/')
print(gcpt_dash_path)

/Users/gem-tah/Desktop/GEM_INFO/GEM_WORK/GCPT-dashboard/data_gcpt/


In [16]:
gcpt_dash_xl = pd.ExcelFile(gcpt_dash_path + gcpt_dash_file)


In [17]:
def gspread_access_file_read_only(key, title):
    """
    key = Google Sheets unique key in the URL
    title = name of the sheet you want to read
    """
    gspread_creds = gspread.oauth(
        scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"],
        credentials_filename=client_secret_full_path,
        # authorized_user_filename=json_token_name,
    )
    gsheets = gspread_creds.open_by_key(key)
    # Access a specific tab
    spreadsheet = gsheets.worksheet(title)
    # expected_header option provided following: https://github.com/burnash/gspread/issues/1007
    df = pd.DataFrame(spreadsheet.get_all_records(expected_headers=[]))
    
    return df

In [18]:
def determine_max_year(gcpt_dash_file):
    yyyy_mm = gcpt_dash_file.split('.xlsx')[0].split(' ')[-1]
    month_int = int(yyyy_mm.split('-')[-1])
    year_int = int(yyyy_mm.split('-')[0])
    if month_int < 6:
        max_year = year_int - 1
    else:
        max_year = year_int
        
    # print(f"max_year: {max_year}")

    return max_year

In [19]:
# create list of countries to choose from (GEM country names)
# data in gcpt_status is most complete; example of Albania, which only has cancelled units, nothing else
max_year = determine_max_year(gcpt_dash_file)
gcpt_status_orig = pd.read_excel(gcpt_dash_xl, sheet_name = f'2014-{max_year}')
gcpt_country_list = gcpt_status_orig['Country'].sort_values().unique().tolist()
gcpt_country_list = ['all'] + gcpt_country_list

### Map (upper left)

In [20]:
def create_country_code_df():
    # TO DO: change below to draw from GEM's country list, which also includes the ISO names
    # "GEM Naming Conventions - Countries & subdivisions"
    # https://docs.google.com/spreadsheets/d/1mtlwSJfWy1gbIwXVgpP3d6CcUEWo2OM0IvPD6yztGXI/edit#gid=0

    # country_codes_file = 'List of ISO 3166 country codes (Wikipedia).xlsx'
    # country_codes_path = '/Users/masoninman/Dropbox/GEM/'
    # country_codes = pd.read_excel(country_codes_path + country_codes_file)


    gem_naming_convention_key = '1mtlwSJfWy1gbIwXVgpP3d6CcUEWo2OM0IvPD6yztGXI'

    # gspread way
    country_codes = gspread_access_file_read_only(gem_naming_convention_key, 'Countries')

    # clean up the codes to remove non-printing characters from wikipedia
    for col in country_codes.columns:
        if country_codes[col].dtype == object:
            country_codes[col] = country_codes[col].str.replace('\xa0', '', regex=False)

    # get rid of parenthetical footnotes at end of names & whitespace
    country_codes['ISO 3166 Country Name'] = country_codes['ISO 3166 Country Name'].str.split('[').str[0].str.strip()

    return country_codes

In [21]:
def create_country_name_conversions_dict():
    """
    Creates a dictionary for converting GEM standard country names to ISO 3166 versions.
    
    This downloads the GEM standard names file from Google Sheets using pygsheets,
    then pares down the df to only those in which GEM uses a different name than ISO 3166.

    Then it creates a dictionary, which can be used for converting from GEM to ISO.
    
    This is needed because Plotly's choropleth function uses ISO names for getting the outline for each country.
    """
    

    standard_country_names_key = '1mtlwSJfWy1gbIwXVgpP3d6CcUEWo2OM0IvPD6yztGXI'

    # gspread way
    df = gspread_access_file_read_only(standard_country_names_key, 'Countries')
    # keep only those with a mismatch
    name_diffs = df.copy()
    name_diffs = name_diffs[name_diffs['GEM name same as ISO 3166?']=='FALSE']
    # exclude those not in ISO
    name_diffs = name_diffs[name_diffs['ISO 3166 Country Name']!='NOT LISTED']
    name_diffs_dict = name_diffs.set_index('GEM Standard Country Name')['ISO 3166 Country Name'].to_dict()   
    

    
    return name_diffs_dict

In [22]:
# get map df
gcpt_map = pd.read_excel(gcpt_dash_xl, sheet_name = 'Map')
gcpt_map = gcpt_map.rename(columns={'Total operating Capacity': 'Capacity (MW)'})

# add any missing countries, to make sure that all countries in gcpt_country_list are in gcpt_map
missing_countries = [x for x in gcpt_country_list if x not in gcpt_map['Country'].tolist()]
if len(missing_countries) > 0:
    print(f"These countries were missing: {missing_countries}")
    missing_df = pd.DataFrame.from_dict({
            'Country': missing_countries, 
            'Capacity (MW)': [float(0)]*len(missing_countries)
        }, orient='columns')
    gcpt_map = pd.concat([gcpt_map, missing_df], sort=False)

# change GEM country names to ISO 3166
name_diffs_dict = create_country_name_conversions_dict()
gcpt_map['ISO 3166 Country Name'] = gcpt_map['Country'].replace(name_diffs_dict)

# Note: Kosovo isn't recognized in ISO 3166, so can't be shown on Plotly map on its own. 
# We could combine it with Serbia for display--but then our data in the dashboard wouldn't be the same as in our spreadsheets & other maps.
# Wikipedia said Kosovo declared independence from Serbia in 2008; it is only partially recognized.

# show countries in gcpt_map not in gcpt_country_list:
extraneous_countries = [x for x in gcpt_map['Country'].tolist() if x not in gcpt_country_list]
if len(extraneous_countries) > 0:
    print(f"Extraneous countries to be removed: {extraneous_countries}")

# keep only countries that are in gcpt_country_list
gcpt_map = gcpt_map[gcpt_map['Country'].isin(gcpt_country_list)]

# merge in ISO country codes (needed by Plotly)
country_codes = create_country_code_df()
gcpt_map = pd.merge(
    country_codes[['ISO 3166 Country Name', 'Country ISO 3166-1 alpha-3']],
    gcpt_map,
    on='ISO 3166 Country Name', 
    how='outer'
)
gcpt_map = gcpt_map.rename(columns={'Country ISO 3166-1 alpha-3': 'iso_alpha'})

# exclude those with no value for iso_alpha
# This excludes notes within the ISO dataset, e.g., "Akrotiri and Dhekelia – See United Kingdom, The."
# Unfortunately, this also excludes Kosovo from GCPT
gcpt_map = gcpt_map[gcpt_map['iso_alpha'].isna()==False]

These countries were missing: ['all']


In [23]:
# if capacity is 0, instead use 1, to avoid zero capacity leading to -inf log value
# similar approach is used in numpy log1p
# https://numpy.org/doc/stable/reference/generated/numpy.log1p.html
gcpt_map['capacity log10 + 1'] = np.log10(gcpt_map['Capacity (MW)'].replace(float(0), float(1)))

# create hover text
# (note: variable hover_text below is a Pandas Series)
hover_text = gcpt_map['Country'] + ': '
hover_text = hover_text + gcpt_map['Capacity (MW)'].map('{:,.0f}'.format) + ' MW'
# hide extra bit, e.g. 'trace 0'; based on https://plotly.com/python/reference/#scatter-hovertemplate
hover_text = hover_text + '<extra></extra>'
gcpt_map['hover_text'] = hover_text

In [24]:
def status_clean_data(df):
    for col in df.columns:
        if col not in ['Country', 'Capacity (MW)']:
            # make the text lowercase
            print(f"Cleaning col {col}")
            df[col] = df[col].str.lower()

    return df

In [25]:
def sort_status(df):
    """
    convert column 'Status' to categorical
    https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html
    """
    
    status_order = [
        'operating',
        'mothballed',
        'announced',
        'pre-permit',
        'permitted',
        'construction',
        'retired',
        'shelved',
        'cancelled',
    ]
    df['Status'] = df['Status'].astype(
        CategoricalDtype(status_order, ordered=False)
    )    
    df = df.sort_values(by=['Country', 'Status', 'Year'])
    
    return df

In [26]:
def status_test_capacity(df, df_init):
    """Runs at end of status_condense, to check that there was no change in the data.
    (Comparing version after condensing the data as it was before condensing the data.)
    
    There are entries with status NaN in the version of the data before condensing,
    because the table prior to that step has a row for each unit, and columns for each year.
    (Other distinguishing features for each unit aren't included in this table.)
    
    If there is no data for a unit for some of the years covered, then we have status NaN for those years.
    (Example: It was proposed only in 2018, then the years 2014-2017 have status NaN.)
    """

    df = df.reset_index()
        
    all_statuses = []
    status_min_year = df['Year'].min()
    for year in range(status_min_year, max_year+1):
        all_statuses += df_init[year].tolist()
    all_statuses = list(set(all_statuses))

    for sel_status in all_statuses:
        print(f"Testing capacities for {sel_status}")
        for sel_year in range(status_min_year, max_year+1):
            sel_init = df_init[df_init[sel_year]==sel_status]
            sel_init_sum = sel_init['Capacity (MW)'].sum()

            # handle two different formats for df
            if 'Status' in df.columns:
                sel = df[(df['Year']==sel_year) & (df['Status']==sel_status)]
                sel_sum = sel['Capacity (MW)'].sum()
                
            else:
                sel = df[df[sel_year]==sel_status]
                sel_sum = sel['Capacity (MW)'].sum()

            # compare values
            abs_diff = abs(sel_sum - sel_init_sum)
            if abs_diff <= 1e-7:
                pass

            elif abs_diff > 1e-7:
                print("Error!" + f" Capacity difference for {sel_year} & {sel_status}.")
                print(f"Initial value: {sel_init_sum}")
                print(f"Current value: {sel_sum}")
            else:
                print("Unexpected case")
    
    # no return

In [27]:
def status_fill_in_missing(df):
    """
    For each country, for each status, fill in zero value if it is missing.
    """
    year_range = range(df['Year'].min(), df['Year'].max() + 1)
    statuses = df['Status'].dropna().unique().tolist()
    for country in df['Country'].unique().tolist():
        for status in statuses:
            for year in year_range:
                df_sel = df[(df['Year']==year) & (df['Country']==country) & (df['Status']==status)]
                if len(df_sel) == 0:
                    # there is a missing value; fill it in
                    # print(f"To fill in missing value for {country}, {year}, {status}") # for db
                    # values in dict below must be lists, to avoid error:
                    # "ValueError: If using all scalar values, you must pass an index"
                    new_df = pd.DataFrame.from_dict({
                                'Country': [country],
                                'Year': [year],
                                'Status': [status],
                                'Capacity (MW)': [float(0)],
                            }, orient='columns')
                    df = pd.concat([df, new_df], sort=False)

    df = df.sort_values(by=['Country', 'Year', 'Status'])
                    
    return df

In [28]:
def status_calculate_global_totals(df):    
    gcpt_status_all = df.groupby(['Year', 'Status'])[['Capacity (MW)']].sum().reset_index()
    gcpt_status_all.insert(0, 'Country', 'all')
    gcpt_status_all = sort_status(gcpt_status_all)
    
    df = pd.concat([gcpt_status_all, df], sort=False)

    return df

In [29]:
def status_condense(df_arg):
    """
    There are sometimes duplicates, for example if two units in a plant have the exact same capacity and history.
    So before setting the index to Country & Capacity, need to get rid of duplicates.
    """
    df = df_arg.copy()
    df_init = df_arg.copy()  # for test
    
    df = df.dropna(how='all')
    
    df = df.set_index(['Country', 'Capacity (MW)'])
    df = df.stack().reset_index()
        
    df = df.rename(columns={'level_2': 'Year', 0: 'Status'})
    
    df = df.groupby(['Country', 'Year', 'Status'])[['Capacity (MW)']].sum().reset_index()
    df = sort_status(df)

    # print(df.columns) # for db
    status_test_capacity(df, df_init)
    
    return df

In [30]:
def test_statuses(gcpt_status_uncondensed):
    accepted_statuses = ['announced', 'cancelled', 'construction', 'mothballed', 'operating', 'permitted',  'pre-permit', 'shelved', 'retired']
    for col in range(2014, max_year +1):
        ser = gcpt_status_uncondensed[col].dropna()
        unaccepted = ser[~ser.isin(accepted_statuses)]
        if len(unaccepted) > 0:
            print(f"Found unaccepted statuses; len(ser): {len(ser)}")
            print(unaccepted.value_counts())
    # no return

In [31]:
gcpt_status_uncondensed = pd.read_excel(gcpt_dash_xl, sheet_name = f'2014-{max_year}')
gcpt_status_uncondensed = status_clean_data(gcpt_status_uncondensed)
test_statuses(gcpt_status_uncondensed)

gcpt_status = status_condense(gcpt_status_uncondensed)
gcpt_status = status_fill_in_missing(gcpt_status)
gcpt_status = status_calculate_global_totals(gcpt_status)

Cleaning col 2014
Cleaning col 2015
Cleaning col 2016
Cleaning col 2017
Cleaning col 2018
Cleaning col 2019
Cleaning col 2020
Cleaning col 2021
Cleaning col 2022
Cleaning col 2023
Testing capacities for permitted
Testing capacities for announced
Testing capacities for pre-permit
Testing capacities for nan
Testing capacities for shelved
Testing capacities for construction
Testing capacities for cancelled
Testing capacities for operating
Testing capacities for mothballed
Testing capacities for retired


### Age & Technology (lower left)
- For bar chart of capacity by age & type
- From the sheet 'Plant type and age'

In [32]:
def age_read_and_clean(gcpt_dash_xl):
    gcpt_age = pd.read_excel(gcpt_dash_xl, sheet_name = 'Plant type and age')
    
    # change capitalization of 'Ultra-Supercritical' to 'Ultra-supercritical'
    gcpt_age['Combustion technology'] = gcpt_age['Combustion technology'].replace(
        'Ultra-Supercritical', 'Ultra-supercritical'
    )
    
    return gcpt_age

def age_condense_data(df):
    print(len(df))

    # bin by decade
    for row in df.index:
        age = df.at[row, 'Plant Age']
        if age < 10:
            df.at[row, 'decade'] = '0-9 years'
        elif age >= 10 and age < 20:
            df.at[row, 'decade'] = '10-19 years'
        elif age >= 20 and age < 30:
            df.at[row, 'decade'] = '20-29 years'
        elif age >= 30 and age < 40:
            df.at[row, 'decade'] = '30-39 years'
        elif age >= 40 and age < 50:
            df.at[row, 'decade'] = '40-49 years'
        elif age >= 50:
            df.at[row, 'decade'] = '50+ years'
        else:
            print("Error!" + f" Issue with age for row {row}: {age}")

    df = df.groupby(['Country', 'decade', 'Combustion technology'])[['Capacity (MW)']].sum()
    df = df.reset_index()
    
    # unstack, then fill in zeros
    df = df.set_index(['Country', 'decade', 'Combustion technology']).unstack()
    df = df.droplevel(0, axis=1)
    
    for col in df.columns:
        df[col] = df[col].fillna(0)

    df = df.reset_index()
    df.columns.name = ''
    
    print(len(df))
    
    return df

def age_calculate_global_totals(df):

    technologies = ['CFB', 'IGCC', 'Subcritical', 'Supercritical', 'Ultra-supercritical', 'Unknown']
    
    gcpt_age_all = df.groupby('decade')[technologies].sum().reset_index()
    gcpt_age_all.insert(0, 'Country', 'all')

    df = pd.concat([gcpt_age_all, df], sort=False)

    return df

def age_fill_in_missing_decades(df):
    decade_list = [
        '0-9 years',
        '10-19 years',
        '20-29 years',
        '30-39 years',
        '40-49 years',
        '50+ years',
    ]
    for country_sel in gcpt_country_list:
        for decade_sel in decade_list:
            df_sel = df[(df['Country']==country_sel) & (df['decade']==decade_sel)]
            if len(df_sel) == 0:
                new_df = pd.DataFrame.from_dict({
                        'Country': country_sel,
                        'decade': decade_sel,
                        'CFB': [float(0)],
                        'IGCC': [float(0)],
                        'Subcritical': [float(0)],
                        'Supercritical': [float(0)],
                        'Ultra-supercritical': [float(0)],
                        'Unknown': [float(0)],
                    }, orient='columns')
                df = pd.concat([df, new_df], sort=False)
            elif len(df_sel) == 1:
                pass

            elif len(df_sel) > 1:
                print("Error!")
            else:
                print("Error! (of another kind)")

    return df


gcpt_age = age_read_and_clean(gcpt_dash_xl)
gcpt_age = age_condense_data(gcpt_age)
gcpt_age = age_calculate_global_totals(gcpt_age)
gcpt_age = age_fill_in_missing_decades(gcpt_age)

6564
245


### Additions & retirements
- data for bar chart additions and retirements (lower-right)
- sheet '2000-2022' (for example)

In [33]:
# data for bar chart additions and retirements (lower-right): sheet '2000-2022' (for example)
# no condensing needed, but need to unstack

additions_max_year = 2023

gcpt_add = pd.read_excel(gcpt_dash_xl, sheet_name = f'2000-{additions_max_year}')


def add_unstack(df):
    df = df.set_index(['Country', 'Year', 'Status'])
    df = df.unstack(-1)
    df = df.droplevel(0, axis=1)
    df = df.reset_index()
    df = df.rename(columns={
        'Operating': 'Added (MW)', 
        'Retired': 'Retired (MW)'
    })
        
    return df

def add_missing_countries(df):
    # add any missing countries, to make sure that all countries in gcpt_country_list are in gcpt_map
    # missing_countries = [x for x in gcpt_country_list if x not in gcpt_add['Country'].tolist()]
    missing_countries = [x for x in gcpt_country_list if x not in gcpt_add['Country'].tolist()]

    print(f"Show any countries missing (which will be added below): {missing_countries}")
    
    for year in range(
        int(gcpt_add['Year'].min()), 
        int(gcpt_add['Year'].max())+1):

        new_df = pd.DataFrame.from_dict({
                'Country': missing_countries,
                'Year': year,
                'Added (MW)': [float(0)]*len(missing_countries),
                'Retired (MW)': [float(0)]*len(missing_countries),
            }, orient='columns')
        df = pd.concat([df, new_df], sort=False)
    return df

def add_calculate_global_totals(df):
    gcpt_add_all = df.groupby('Year')[['Added (MW)', 'Retired (MW)', 'Net added (MW)']].sum().reset_index()
    gcpt_add_all.insert(0, 'Country', 'all')
    df = pd.concat([gcpt_add_all, df], sort=False)
    
    return df


In [34]:
gcpt_add = add_unstack(gcpt_add)

gcpt_add = add_missing_countries(gcpt_add)

gcpt_add['Net added (MW)'] = gcpt_add['Added (MW)'].sub(gcpt_add['Retired (MW)'])
gcpt_add = add_calculate_global_totals(gcpt_add)

Show any countries missing (which will be added below): ['all']


In [35]:

# TEST: check that there are not any rows with NaNs for 'Net added'
test = gcpt_add[gcpt_add['Net added (MW)'].isna()]
if len(test) > 0:
    print("Error!" + f" There were rows with value NaN for 'Net added (MW)'")

In [36]:

# export to various sheets of one Excel file by creating ExcelWriter object
template_name = gcpt_dash_file.split('.xlsx')[0]
save_timestamp = time.strftime('%Y-%m-%d_%H%M', time.localtime())
file_name = f'{template_name} - processed for Dash {save_timestamp}.xlsx'
with pd.ExcelWriter(gcpt_dash_path + file_name) as writer:
    gcpt_map.to_excel(writer, sheet_name='map', index=False)  
    gcpt_status.to_excel(writer, sheet_name='status', index=False)
    gcpt_age.to_excel(writer, sheet_name='age', index=False)
    gcpt_add.to_excel(writer, sheet_name='additions', index=False)
    print(f"Saved to file: {file_name}")

Saved to file: GCPT dashboard data 2023-08 - processed for Dash 2023-09-18_1649.xlsx
